In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

## 依照指示取出模型特定層的資訊

In [2]:
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16* 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()


### 取出 self.pool層兩次的輸出，包含：
* x = self.pool(F.relu(self.conv1(x)))
* x = self.pool(F.relu(self.conv2(x)))

In [3]:
outputs= []
def layer1_hook(module, input_, output):
    outputs.append(output)
model.pool.register_forward_hook(layer1_hook)

In [4]:
input_ = torch.randn(1, 3, 32, 32)
output = model(input_)

In [5]:
outputs

[tensor([[[[1.0250e+00, 7.7583e-01, 5.7529e-01,  ..., 1.5886e-01,
            2.9134e-02, 8.2030e-01],
           [1.0341e+00, 4.9772e-01, 1.2255e-01,  ..., 7.0766e-01,
            7.3284e-01, 8.3083e-01],
           [1.4643e+00, 5.9718e-01, 6.5815e-01,  ..., 4.7446e-01,
            1.1583e+00, 7.6607e-01],
           ...,
           [3.0558e-01, 3.8262e-01, 3.8806e-01,  ..., 5.4402e-01,
            6.4012e-01, 4.6095e-01],
           [8.7970e-01, 2.0105e-01, 5.0638e-01,  ..., 3.4169e-01,
            5.5235e-01, 6.5048e-01],
           [1.3811e-01, 6.6971e-01, 8.5055e-01,  ..., 5.3767e-01,
            9.2830e-01, 7.6516e-01]],
 
          [[1.2049e-01, 7.3529e-01, 1.3644e+00,  ..., 7.2438e-01,
            1.1366e+00, 6.7878e-01],
           [1.7332e+00, 4.3479e-01, 6.0437e-01,  ..., 0.0000e+00,
            9.2381e-01, 5.4424e-01],
           [8.3101e-01, 4.1275e-01, 4.7992e-02,  ..., 1.0580e+00,
            6.8643e-01, 1.2178e+00],
           ...,
           [3.5774e-01, 8.8580e-04, 1.

In [6]:
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])


## 加入自定義 initialization fuction

#### 對所有Conv2D層使用自定義initialization function
* weight : nn.init.kaiming_normal_
* bias : 全部輸入1

In [7]:
from torch.nn import init

def weights_init(m):
    #classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d):
        #torch.nn.init.xavier_uniform_(m.weight)
        nn.init.kaiming_normal_(m.weight.data,
                                    a=0,
                                    mode='fan_out',
                                    nonlinearity='relu')
        if m.bias is not None:
            m.bias.data.fill_(1)
            
model.apply(weights_init)

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

#### 查看 conv層的bias是否皆為1

In [8]:
for name, parameters in model.named_parameters():
    if ('conv' in name) and ('bias' in name):
        print(name, parameters)
        print('\n')

conv1.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)


conv2.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)


